Martin Paz

### TP3 Visión por computadora


Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en Material_TPs/TP3/images a partir del template Material_TPs/TP3/template

1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos
2. (4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen coca_multi.png con el mismo témplate del ítem 2
3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes

Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza de la detección

#### Punto 1

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
def load_images(image_paths):
    images = {}
    for path in image_paths:
        image_name = path.split('/')[-1].split('.')[0]
        images[image_name] = cv.imread(path)
    return images

# Definimos las rutas
image_paths = [
    'images/coca_logo_1.png',
    'images/coca_logo_2.png',
    'images/COCA-COLA-LOGO.jpg',
    'images/logo_1.png',
    'images/coca_multi.png',
    'images/coca_retro_1.png',
    'images/coca_retro_2.png'
]

# Cargamos las imagenes
template = cv.imread('template/pattern.png')
images = load_images(image_paths)

In [ ]:
# Graficamos
plt.figure(figsize=(20, 15))
for i, (name, img) in enumerate(images.items()):
    plt.subplot(3, 3, i+1)
    plt.imshow(img[:,:,::-1])
    plt.axis('off')
    plt.title(name)

plt.tight_layout()
plt.show()

1. (4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos

In [4]:
# Utilizo template matching para encontrar el patron dentro de la imagen
# Analizo en multiples escalas del patron
def findTemplate(template, img, n, method=cv.TM_CCOEFF_NORMED):
    
    # Convierto las imagenes a escala de grises
    gray_template  = cv.cvtColor(template, cv.COLOR_BGR2GRAY) 
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
    
    # Obtengo una imagen de bordes para abstraernos del fondo y colores    
    filter_gray_img = cv.Laplacian(cv.GaussianBlur(gray_img,(5,5),0), cv.CV_8U)
    
    ph,pw = gray_template.shape
    ih,iw = filter_gray_img.shape
    
    # Analizo n escalas distintas del patron
    # Utilizo el ancho de la imagen como ancho del patron mas grande
    scales = np.linspace(0, iw/pw, n+1)[1:]
    
    # Guardo los resultados de todas las escalas en una lista
    matching = list()
    minMaxRes = np.zeros((n,8))
    
    for i, scale in enumerate(scales):
        
        # Escalo el patron
        resTemplate = cv.resize(gray_template, None, fx=scale, fy=scale)
        
        # Aplico un filtro laplaciano para quedarme con los bordes
        resTemplate = cv.Laplacian(cv.GaussianBlur(resTemplate,(3,3),0), cv.CV_8U)
        ph,pw = resTemplate.shape
        
        # Aplico el template matching 
        res = cv.matchTemplate(filter_gray_img, resTemplate, method)
         
        # Hago una estandarizacion para poder hacer una comparacion entre todos los resultados 
        epsilon = 1e-10  # Small value to avoid division by zero
        res = (res - np.mean(res)) / (np.std(res) + epsilon)
        
        # Guardo el resultado
        matching.append(res)

        # Guardo datos de interes para graficar la bounding box 
        minVal, maxVal, minLoc, maxLoc = cv.minMaxLoc(res)
        minMaxRes[i,:] = [minVal, maxVal, minLoc[0], minLoc[1], maxLoc[0], maxLoc[1], ph, pw]
    
    
    # Busco el maximo entre todas las escalas <- se podria generalizar para los metodos que minimizan
    iMax = np.argmax(minMaxRes[:,1])
    loc = minMaxRes[iMax,:]
    
    # Grafico una bounding box donde encontre el maximo
    matchImg = img.copy()
    topLeft = (int(loc[4]),int(loc[5]))
    bottomRight = (int(loc[4]+loc[7]), int(loc[5]+loc[6]))
 
    cv.rectangle(matchImg, topLeft, bottomRight, (0,255,0), 2)    
  
    return matching, minMaxRes, matchImg

In [ ]:
# Número de imágenes
num_images = len(images)

# Calcular el número de filas necesarias
num_rows = (num_images + 1) // 2  # +1 para redondear hacia arriba si no es divisible por 2

# Crear una figura con subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(20, 10 * num_rows))

# Aplanar el array de ejes para facilitar la iteración
axes = axes.flatten()

# Iterar sobre las imágenes y mostrar cada una en un subplot
for i, img in enumerate(images):
    _, _, res = findTemplate(template, images[img], 10)
    # Mostrar la imagen con la detección y el nombre
    axes[i].imshow(cv.cvtColor(res, cv.COLOR_BGR2RGB))
    axes[i].set_title(img)
    axes[i].axis('off')

# Ocultar los subplots vacíos si hay menos imágenes que subplots
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

# Ajustar el layout para que no haya solapamientos
plt.tight_layout()
plt.show()

Cambiamos los valores de n para detectar en los casos donde  coca_logo_2 y coca_multi falló!

In [ ]:
_, _, res = findTemplate(template, images['coca_logo_2'], 1)
plt.figure(figsize=(8,6))
plt.imshow(res[:,:,::-1])
plt.tight_layout()


In [ ]:
_, _, res = findTemplate(template, images['coca_multi'], 9)
plt.figure(figsize=(8,6))
plt.imshow(res[:,:,::-1])
plt.tight_layout()


#### Vamos a ver detalladamente que encuentra en cada imagen y porque falla para otros n. Color verde para el valor mas alto, negro para los demas

In [8]:
def findTemplateDetailed(template, img, n, method=cv.TM_CCOEFF_NORMED):
    # Convertir la plantilla y la imagen a escala de grises
    gray_template = cv.cvtColor(template, cv.COLOR_BGR2GRAY)
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Aplicar un filtro Laplaciano a la imagen gris después de un desenfoque Gaussiano
    filter_gray_img = cv.Laplacian(cv.GaussianBlur(gray_img, (5, 5), 0), cv.CV_8U)

    # Obtener las dimensiones de la plantilla y la imagen filtrada
    ph, pw = gray_template.shape
    ih, iw = filter_gray_img.shape

    # Generar una lista de escalas para redimensionar la plantilla
    scales = np.linspace(0, iw / pw, n + 1)[1:]
    matching = list()
    minMaxRes = np.zeros((n, 8))
    images_with_boxes = []

    # Iterar sobre cada escala
    for i, scale in enumerate(scales):
        # Redimensionar la plantilla y aplicar un filtro Laplaciano después de un desenfoque Gaussiano
        resTemplate = cv.resize(gray_template, None, fx=scale, fy=scale)
        resTemplate = cv.Laplacian(cv.GaussianBlur(resTemplate, (3, 3), 0), cv.CV_8U)
        ph, pw = resTemplate.shape

        # Realizar la coincidencia de plantillas
        res = cv.matchTemplate(filter_gray_img, resTemplate, method)
        epsilon = 1e-10
        res = (res - np.mean(res)) / (np.std(res) + epsilon)

        # Almacenar los resultados de la coincidencia
        matching.append(res)
        minVal, maxVal, minLoc, maxLoc = cv.minMaxLoc(res)
        minMaxRes[i, :] = [minVal, maxVal, minLoc[0], minLoc[1], maxLoc[0], maxLoc[1], ph, pw]

        # Dibujar un rectángulo en la imagen original en la posición de la mejor coincidencia
        img_with_box = img.copy()
        topLeft = (int(maxLoc[0]), int(maxLoc[1]))
        bottomRight = (int(maxLoc[0] + pw), int(maxLoc[1] + ph))
        color = (0, 0, 0)  # Negro para valores más bajos
        cv.rectangle(img_with_box, topLeft, bottomRight, color, 2)
        cv.putText(img_with_box, f'Scale: {i+1}, Value: {maxVal:.2f}', (topLeft[0], topLeft[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        images_with_boxes.append(img_with_box)

    # Encontrar la escala con el valor máximo de coincidencia
    iMax = np.argmax(minMaxRes[:, 1])
    loc = minMaxRes[iMax, :]

    # Dibujar un rectángulo verde en la imagen original en la posición de la mejor coincidencia
    matchImg = img.copy()
    topLeft = (int(loc[4]), int(loc[5]))
    bottomRight = (int(loc[4] + loc[7]), int(loc[5] + loc[6]))
    cv.rectangle(matchImg, topLeft, bottomRight, (0, 255, 0), 2)  # Verde para el valor más alto
    cv.putText(matchImg, f'Scale: {iMax+1}, Value: {loc[1]:.2f}', (topLeft[0], topLeft[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    images_with_boxes[iMax] = matchImg

    return matching, minMaxRes, matchImg, images_with_boxes

def plot_images_with_boxes(images_with_boxes):
    # Calcular el número de filas necesarias (2 imágenes por fila)
    num_rows = (len(images_with_boxes) + 1) // 2

    # Crear una figura con subplots
    fig, axes = plt.subplots(num_rows, 2, figsize=(20, 10 * num_rows))

    # Aplanar el array de ejes para facilitar la iteración
    axes = axes.flatten()

    # Iterar sobre las imágenes y mostrar cada una en un subplot
    for i, img_with_box in enumerate(images_with_boxes):
        axes[i].imshow(cv.cvtColor(img_with_box, cv.COLOR_BGR2RGB))
        axes[i].set_title(f'Escala {i+1}')
        axes[i].axis('off')

    # Ocultar los subplots vacíos si hay menos imágenes que subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    # Ajustar el layout para evitar solapamientos
    plt.tight_layout()
    plt.show()

In [ ]:
_, _, _, images_with_boxes = findTemplateDetailed(template, images['coca_logo_2'], 10)

plot_images_with_boxes(images_with_boxes)

In [ ]:
_, _, _, images_with_boxes = findTemplateDetailed(template, images['coca_multi'], 10)

plot_images_with_boxes(images_with_boxes)

#### Punto 2.

In [11]:
# Utilizo template matching para encontrar el patron dentro de la imagen
# Analizo en multiples escalas del patron y obtengo multiples detecciones
def findMultipleTemplate(template, img, n, th, method=cv.TM_CCOEFF_NORMED, nms=False):
    
    # Convierto las imagenes a escala de grises
    templateGray = cv.cvtColor(template, cv.COLOR_BGR2GRAY) 
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
    
    # Obtengo una imagen de bordes para abstraernos del fondo y colores    
    imgGray = cv.Laplacian(cv.GaussianBlur(imgGray,(5,5),0), cv.CV_8U)
    
    ph,pw = templateGray.shape
    ih,iw = imgGray.shape
    
    # Analizo n escalas distintas del patron
    # Utilizo el ancho de la imagen como ancho del patron mas grande
    scales = np.linspace(0, iw/pw, n+1)[1:]
    
    # Guardo los resultados de todas las escalas en una lista
    matching = list()

    # Guardo la ubicacion de los match para luego hacer Non Maximun Suppression
    locRes = list()
    confidenceScores = list()
    values_list = []
    
    matchImg = img.copy()
    
    for i, scale in enumerate(scales):
        
        # Escalo el patron
        resTemplate = cv.resize(templateGray, None, fx=scale, fy=scale)
        
        # Aplico un filtro laplaciano para quedarme con los bordes
        resTemplate = cv.Laplacian(cv.GaussianBlur(resTemplate,(3,3),0), cv.CV_8U)
        ph,pw = resTemplate.shape
        
        # Aplico el template matching 
        res = cv.matchTemplate(imgGray, resTemplate, method)
        
        # Hago una estandarizacion para poder hacer una comparacion entre todos los resultados 
        epsilon = 1e-10  # Small value to avoid division by zero
        res = (res - np.mean(res)) / (np.std(res) + epsilon)
        
        # Guardo el resultado
        matching.append(res)
        
        # Si el valor maximo supera un umbral th la considero como un match
        # Guardo datos de interes para graficar las bounding box
        # Guardo coordenadas [x0, y0, x1, y1]
        loc = np.where(res >= th)
        for pt in zip(*loc[::-1]): 
            
            locRes.append([pt[0], pt[1], pt[0]+pw, pt[1]+ph])
            
            # Obtengo una metrica del nivel de confianza de la deteccion
            value = res[pt[::-1]]
            values_list.append(value)
            confidenceScores.append(value/np.max(res))
            
        # Dibujo un rectangulo en la imagen original
        for coord in locRes:
            cv.rectangle(matchImg, (coord[0],coord[1]), (coord[2], coord[3]), (0,255,0), 2)
    
    return matching,  matchImg

In [ ]:
_, res = findMultipleTemplate(template, images['coca_multi'], 15, 3.2, cv.TM_CCOEFF_NORMED)

plt.figure(figsize=(16,10))
plt.imshow(res[:,:,::-1])
plt.tight_layout()

In [ ]:
_, res = findMultipleTemplate(template, images['coca_multi'], 13, 3.5, cv.TM_CCORR)

plt.figure(figsize=(16,10))
plt.imshow(res[:,:,::-1])
plt.tight_layout()

#### Se llega a identificar la plantilla con el metodo TM_CCORR, esto puede ser debido a que la plantilla y la imagen tienen valores similares con diferentes intensidades, ademas la iluminación puede no ser significativa.

3. (2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes

In [ ]:

# Graficar las imágenes con las detecciones
plt.figure(figsize=(20, 15))
image_names = ['coca_logo_1', 'coca_logo_2', 'coca_multi', 'coca_retro_1', 'coca_retro_2', 'COCA-COLA-LOGO', 'logo_1']
num_scales = [1, 1, 13, 7, 5, 1, 3]
thresholds = [1, 1, 3.5, 4, 7, 1, 3.5]

for i, (name, num_scale, threshold) in enumerate(zip(image_names, num_scales, thresholds)):
    image = images[name]
    _, result_image = findMultipleTemplate(template, image, num_scale, threshold, method=cv.TM_CCORR, nms=True)
    plt.subplot(3, 3, i + 1)
    plt.imshow(result_image[:, :, ::-1])
    plt.axis('off')
    plt.title(name)

plt.tight_layout()
plt.show()



##### Nivel de confianza: para cada coincidencia detectada, se calcula el nivel de confianza como el valor de la respuesta en esa posición dividido por el valor máximo de la respuesta:

In [ ]:
# Función para aplicar Non-Maximum Suppression (NMS) y eliminar rectángulos superpuestos
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]

        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int")

def findMultipleTemplateWithConfidence(template, img, n, th, method=cv.TM_CCORR, nms=True, overlapThresh=0.3):
    # Convierto las imagenes a escala de grises
    templateGray = cv.cvtColor(template, cv.COLOR_BGR2GRAY) 
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) 
    
    # Obtengo una imagen de bordes para abstraernos del fondo y colores    
    imgGray = cv.Laplacian(cv.GaussianBlur(imgGray,(5,5),0), cv.CV_8U)
    
    ph, pw = templateGray.shape
    ih, iw = imgGray.shape
    
    # Analizo n escalas distintas del patron
    # Utilizo el ancho de la imagen como ancho del patron mas grande
    scales = np.linspace(0, iw/pw, n+1)[1:]
    
    # Guardo los resultados de todas las escalas en una lista
    matching = list()

    # Guardo la ubicacion de los match para luego hacer Non Maximun Suppression
    locRes = list()
    confidenceScores = list()
    values_list = []
    
    matchImg = img.copy()
    
    for i, scale in enumerate(scales):
        # Escalo el patron
        resTemplate = cv.resize(templateGray, None, fx=scale, fy=scale)
        
        # Aplico un filtro laplaciano para quedarme con los bordes
        resTemplate = cv.Laplacian(cv.GaussianBlur(resTemplate,(3,3),0), cv.CV_8U)
        ph, pw = resTemplate.shape
        
        # Aplico el template matching 
        res = cv.matchTemplate(imgGray, resTemplate, method)
        
        # Hago una estandarizacion para poder hacer una comparacion entre todos los resultados 
        epsilon = 1e-10  # Small value to avoid division by zero
        res = (res - np.mean(res)) / (np.std(res) + epsilon)
        
        # Guardo el resultado
        matching.append(res)
        
        # Si el valor maximo supera un umbral th la considero como un match
        # Guardo datos de interes para graficar las bounding box
        # Guardo coordenadas [x0, y0, x1, y1]
        loc = np.where(res >= th)
        for pt in zip(*loc[::-1]): 
            locRes.append([pt[0], pt[1], pt[0]+pw, pt[1]+ph])
            
            # Obtengo una metrica del nivel de confianza de la deteccion
            value = res[pt[::-1]]
            values_list.append(value)
            confidenceScores.append(value/np.max(res))
    
    # Aplicar Non-Maximum Suppression si está habilitado
    if nms:
        locRes = np.array(locRes)
        locRes = non_max_suppression_fast(locRes, overlapThresh)
    
    # Dibujo los rectángulos y agrego el valor de confianza en la imagen original
    for i, coord in enumerate(locRes):
        cv.rectangle(matchImg, (coord[0], coord[1]), (coord[2], coord[3]), (0, 255, 0), 2)
        confidence = confidenceScores[i]
        cv.putText(matchImg, f'{confidence:.2f}', (coord[0], coord[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    return matching, matchImg


img = images['coca_multi']  # Imagen en la que buscar

# Llamar a la función findMultipleTemplate
matching, matchImg = findMultipleTemplateWithConfidence(template, img, 13, 3.5)

# Mostrar la imagen con las detecciones
plt.imshow(cv.cvtColor(matchImg, cv.COLOR_BGR2RGB))
plt.title('Detecciones con NMS y Confianza')
plt.show()